<a href="https://colab.research.google.com/github/sirjosev/IndividualProject_VoiceCloneForTextToSpeech/blob/main/Voice_Cloning_for_Text_to_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Noé Tits - Numediart (UMONS) - [noetits.com](https://noetits.com)

# Install

In [ ]:
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 2306 (delta 3), reused 6 (delta 2), pack-reused 2286
Receiving objects: 100% (2306/2306), 360.24 MiB | 30.15 MiB/s, done.
Resolving deltas: 100% (1252/1252), done.


In [ ]:
!cd Real-Time-Voice-Cloning && pip install -r requirements.txt

     |████████████████████████████████| 377.0MB 57kB/s 
     |████████████████████████████████| 1.4MB 34.4MB/s 
     |████████████████████████████████| 71kB 21.3MB/s 
     |████████████████████████████████| 62.1MB 709kB/s 
     |████████████████████████████████| 204kB 36.8MB/s 
     |████████████████████████████████| 71kB 23.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ee/87/ce/a5023722374ca73b57fc8d4284ba6f973c01219b3c385a07e0
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom webrtcvad torchfile


In [ ]:
!pip install torch torchvision

In [ ]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [ ]:
# A function to download files from google drive url
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)


In [ ]:
# Download pretrained model. The id comes from the google drive url
id='1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc'
destination='pretrained.zip'
download_file_from_google_drive(id, destination)

In [ ]:
!unzip ./pretrained.zip -d Real-Time-Voice-Cloning/

Archive:  ./pretrained.zip
   creating: Real-Time-Voice-Cloning/encoder/saved_models/
  inflating: Real-Time-Voice-Cloning/encoder/saved_models/pretrained.pt  
   creating: Real-Time-Voice-Cloning/synthesizer/saved_models/
   creating: Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/
   creating: Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: Real-Time-Voice-Cloning/vocoder/saved_models/
   creating: Real-Time-Voice-Cloning/

# Voice Cloning

In [ ]:
import os
os.chdir('Real-Time-Voice-Cloning')

In [ ]:
# This is to test if the setup is working
!python demo_cli.py --no_sound

W0710 13:50:51.912228 139798440265600 deprecation_wrapper.py:119] From /content/Real-Time-Voice-Cloning/synthesizer/models/modules.py:91: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

Arguments:
    enc_model_fpath:   encoder/saved_models/pretrained.pt
    syn_model_dir:     synthesizer/saved_models/logs-pretrained
    voc_model_fpath:   vocoder/saved_models/pretrained/pretrained.pt
    low_mem:           False
    no_sound:          True

Running a test of your configuration...

Found 1 GPUs available. Using GPU 0 (Tesla K80) of compute capability 3.7 with 12.0Gb total memory.

Preparing the encoder, the synthesizer and the vocoder...
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt
Testing your configuration with small inputs.
	Testing the encoder...
	Tes

In [ ]:
from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import argparse
import torch
import sys

In [ ]:
enc_model_fpath=Path("encoder/saved_models/pretrained.pt")
syn_model_dir=Path("synthesizer/saved_models/logs-pretrained/")
voc_model_fpath=Path("vocoder/saved_models/pretrained/pretrained.pt")


In [ ]:
encoder.load_model(enc_model_fpath)
synthesizer = Synthesizer(syn_model_dir.joinpath("taco_pretrained"))
vocoder.load_model(voc_model_fpath)

Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


La cellule suivante permet de charger un fichier audio d'un enregistrement de la voix que vous souhaitez cloner.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 2902_9008_000054_000002.wav to 2902_9008_000054_000002.wav
User uploaded file "2902_9008_000054_000002.wav" with length 375882 bytes


In [ ]:
list(uploaded.keys())[0]

'2902_9008_000054_000002.wav'

In [ ]:
import librosa
ref_sentence,fs=librosa.load(list(uploaded.keys())[0],sr=16000)

In [ ]:
from IPython.display import Audio
Audio(ref_sentence,rate=fs,autoplay=True)

La synthèse semble mieux fonctionner quand les phrases sont longues avec ce modèle. En faisant des essais avec une courte phrases, il y a des bruits sourds générés au milieu.

In [ ]:
embed=encoder.embed_utterance(ref_sentence)
embed /= np.linalg.norm(embed)
mels = synthesizer.synthesize_spectrograms(['It is likely that the ability of the proposed model to generalize well across a wide variety of speakers is based on the quality of the representation learned by the speaker encoder.'], [embed])
mel = np.concatenate(mels, axis=1)

In [ ]:
#wav_syn=vocoder.infer_waveform(mel)

# Slower, but better quality when batched is False
wav_syn=vocoder.infer_waveform(mel, batched=False)

{| ████████████████ 121500/121600 | Batch Size: 1 | Gen Rate: 0.8kHz | }

In [ ]:
Audio(wav_syn,rate=fs,autoplay=True)